# Agrupamento Isolation Forest - Experimento

Este é um componente que treina um modelo Isolation Forest usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para ajustes de modelos, pré-processamento de dados, seleção e avaliação de modelos, além de outras funcionalidades.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [ ]:
# parâmetros
dataset = "/tmp/data/iris.csv" #@param {type:"string"}

# selected features to perform the model
filter_type = "remover" #@param ["incluir","remover"] {type:"string",multiple:false,label:"Modo de seleção das features",description:"Se deseja informar quais features deseja incluir no modelo, selecione a opção 'incluir'. Caso deseje informar as features que não devem ser utilizadas, selecione 'remover'. "}
model_features = "" #@param {type:"feature",multiple:true,label:"Features para incluir/remover no modelo",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}

# features to apply Ordinal Encoder
ordinal_features = "" #@param {type:"feature",multiple:true,label:"Features para fazer codificação ordinal", description: "Seu modelo utilizará a codificação ordinal para as features selecionadas. As demais features categóricas serão codificadas utilizando One-Hot-Encoding."}

# hyperparameters
max_samples = "auto" #@param {type:"float", label:"Quantidade máxima de amostras", description:"O número de amostras máximo a serem retiradas dos dados para treinar cada estimador de base."}
contamination = 0.1 #@param {type:"float", label:"Contaminação", description:"A quantidade de contaminação do conjunto de dados, ou seja, a proporção de outliers no conjunto de dados."}
max_features = 1.0 #@param {type:"float", label:"Quantidade máxima de características", description:"O número de características máximo a serem retiradas dos dados para treinar cada estimador de base."}


# Plots to ignore
plots_ignore = [""] #@param ["Dados de Teste", "Tabelas de Dados"] {type:"string",multiple:true,label:"Gráficos a serem ignorados", description: "Diversos gráficos são gerados ao executar o treinamento e validação do modelo, selecione quais não devem ser gerados."}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

featuretypes = np.array(featuretypes)
columns = df.columns.to_numpy()

## Filtragem das features 

Seleciona apenas as features que foram declaradas no parâmetro model_features. Se nenhuma feature for especificada, todo o conjunto de dados será utilizado para a modelagem.

In [ ]:
columns_to_filter = columns

if len(model_features) >= 1:

    if filter_type == "incluir":
        columns_index = (np.where(np.isin(columns, model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        featuretypes = featuretypes[columns_index]
    else:
        columns_index = (np.where(np.isin(columns, model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns, columns_index)
        featuretypes = np.delete(featuretypes, columns_index)

# keep the features selected
df_model = df[columns_to_filter]

## Configuração dos atributos

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

# Selects non-numerical features to apply ordinal encoder or one-hot encoder
ordinal_features = np.array(ordinal_features)

non_numerical_indexes_ordinal = np.where(
    ~(featuretypes == NUMERICAL) & np.isin(columns_to_filter, ordinal_features)
)[0]

non_numerical_indexes_one_hot = np.where(
    ~(featuretypes == NUMERICAL) & ~(np.isin(columns_to_filter, ordinal_features))
)[0]

# After the step handle_missing_values,
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = np.arange(len(numerical_indexes))

non_numerical_indexes_after_handle_missing_values = np.arange(
    len(numerical_indexes), len(featuretypes)
)

ordinal_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[
    np.where(np.isin(non_numerical_indexes, non_numerical_indexes_ordinal))[0]
]

one_hot_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[
    np.where(np.isin(non_numerical_indexes, non_numerical_indexes_one_hot))[0]
]

## Dividindo o conjunto de dados em treino e teste

Conjunto de treino: usado para ajustar o modelo.
Conjunto de teste: usada para fornecer uma avaliação imparcial de um ajuste de modelo no conjunto de dados de treinamento.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df_model, train_size=0.7)

## Treina modelo usando sklearn.ensemble.IsolationForest

In [ ]:
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    steps=[
        (
            "handle_missing_values",
            ColumnTransformer(
                [
                    ("imputer_mean", SimpleImputer(strategy="mean"), numerical_indexes),
                    (
                        "imputer_mode",
                        SimpleImputer(strategy="most_frequent"),
                        non_numerical_indexes,
                    ),
                ],
                remainder="drop",
            ),
        ),
        (
            "handle_categorical_features",
            ColumnTransformer(
                [
                    (
                        "feature_encoder_ordinal",
                        OrdinalEncoder(),
                        ordinal_indexes_after_handle_missing_values,
                    ),
                    (
                        "feature_encoder_onehot",
                        OneHotEncoder(),
                        one_hot_indexes_after_handle_missing_values,
                    ),
                ],
                remainder="passthrough",
            ),
        ),
        (
            "estimator",
            IsolationForest(
                max_samples=max_samples,
                contamination=contamination,
                max_features=max_features,
            ),
        ),
    ]
)

pipeline.fit(X_train)

features_after_pipeline = np.concatenate(
    (columns_to_filter[numerical_indexes], columns_to_filter[non_numerical_indexes])
)

## Avalia desempenho

No caso do Isolation Florest, podemos medir o desempenho obtendo a anomalia média.

In [ ]:
new_columns = ["Anomaly"]

y_pred = pipeline.predict(X_test)
score = pipeline.predict(df_model)

df_model["Anomaly"] = score

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [ ]:
# from platiagro import save_metrics

# save_metrics(anomaly_score=y_pred)
import mlflow

with mlflow.start_run():
    mlflow.sklearn.autolog()
    mlflow.log_param("anomaly_score", y_pred)

## Visualiza resultados

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from platiagro.plotting import plot_clustering_data

if "Dados de Teste" not in plots_ignore:
    
    ax = plot_clustering_data(pipeline, columns, X_test, y_pred)
    plt.show()

Cria visualização do resultado como uma planilha.

In [ ]:
from platiagro.plotting import plot_data_table

if "Tabelas de Dados" not in plots_ignore:

    ax = plot_data_table(df_model.copy())
    plt.show()

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
# save dataset changes
df_model.to_csv(dataset, index=False)

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "pipeline": pipeline,
    "columns": columns,
    "columns_to_filter": columns_to_filter,
    "new_columns": new_columns,
    "features_after_pipeline": features_after_pipeline,
}

dump(artifacts, "/tmp/data/isolation-forest-clustering.joblib")